# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('total number of rows: '+str(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print('list of event data rows:')
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


total number of rows: 8056


In [7]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [11]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [13]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify_ks 
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [14]:
session.set_keyspace('sparkify_ks')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
# session_id as the partition key
# item_in_session as the clustering key
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfos_by_session_item (
            session_id int,
            item_in_session int,
            artist varchar,
            song varchar,
            length FLOAT,
            PRIMARY KEY (session_id, item_in_session));
    """)
except Exception as e:
    print(e)

In [29]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songinfos_by_session_item (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [49]:
try:
    query = "select artist, song, length from songinfos_by_session_item WHERE session_id = 338 and item_in_session = 4"
    rows = session.execute(query)
    df = pd.DataFrame(rows._current_rows,columns=['artist','song','length'])
except Exception as e:
    print(e)

df 

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [53]:
#Query 2
# user_id and session_id as the primary key and session_id as the partition key
# item_in_session as the clustering key
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_infos (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song TEXT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY ((user_id, session_id), item_in_session));
    """)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_infos (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query,(int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

try:
    query = "select artist, song, first_name, last_name from user_infos WHERE user_id = 10 and session_id = 182"
    rows = session.execute(query)
    df = pd.DataFrame(rows._current_rows,columns=['artist', 'song', 'first_name', 'last_name'])
except Exception as e:
    print(e)

df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [62]:
#Query 3
# song, first_name, last_name, userId as the primary key
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_infos_by_song (
            song varchar,
            first_name varchar,
            last_name varchar,
            userId INT,
            PRIMARY KEY (song, first_name, last_name, userId));
    """)
except Exception as e:
    print(e)     

In [63]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_infos_by_song (song, first_name, last_name, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query,(line[9], line[1], line[4],int(line[10])))

In [64]:
try:
    query = "select first_name, last_name from user_infos_by_song WHERE song = 'All Hands Against His Own'"
    rows = session.execute(query)
    df = pd.DataFrame(rows._current_rows,columns=['first_name', 'last_name'])
except Exception as e:
    print(e)

df

,first_name,last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [65]:
query = "DROP TABLE IF EXISTS songinfos_by_session_item"
try:
    session.execute(query)
    print("DROP TABLE songinfos_by_session_item ok.")
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_infos"
try:
    session.execute(query)
    print("DROP TABLE user_infos ok.")
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_infos_by_song"
try:
    session.execute(query)
    print("DROP TABLE user_infos_by_song ok.")
except Exception as e:
    print(e)

DROP TABLE songinfos_by_session_item ok.
DROP TABLE user_infos ok.
DROP TABLE user_infos_by_song ok.


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()